In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

import emachine as EM

In [2]:
np.random.seed(1)

In [3]:
s0 = np.loadtxt('s_test.dat')
l,n = s0.shape
print(l,n)

1102 36


In [4]:
# find conserved variables
fc = 0.9
l,n = s0.shape
frequency = [max(np.unique(s0[:,i], return_counts=True)[1]) for i in range(n)]
cols_conserved = [i for i in range(n) if frequency[i]/float(l) > fc]
print(cols_conserved)

[6, 16, 31, 34]


In [5]:
cols_active = np.delete(np.arange(0,n),cols_conserved)
print(len(cols_active))

32


In [6]:
# load predicted w from train set
#w_mle = np.loadtxt('w_mle.dat')
w_ple = np.loadtxt('w_ple.dat')
w_em = np.loadtxt('w_em.dat')
print(w_em.shape)

(666,)


In [7]:
# random missing position
for n_hidden in [2,4,6,8,10,12]:
#for n_hidden in [2]:
    # every possibilities of configurations of hiddens
    s_hidden_possibles = np.asarray(list(itertools.product([1.0, -1.0], repeat=n_hidden)))
    n_possibles = s_hidden_possibles.shape[0]
    #print(n_possibles)

    # consider a specific sample t:
    acc = np.zeros((3,l))
    for t in range(l):
        s = s0[t]
        #print(s)

        #hidden = np.random.choice(np.arange(17,n),n_hidden,replace=False)
        #hidden = np.random.choice(np.arange(0,n),n_hidden,replace=False)
        
        hidden = np.random.choice(cols_active,n_hidden,replace=False)       
        #print(hidden)

        
        s_possibles = np.tile(s,(n_possibles,1))
        s_possibles[:,hidden] = s_hidden_possibles
        #print(s_possibles)

        # calculate energy of each possible configuration
        ops = EM.operators(s_possibles)

        """
        #----------------------------------------------
        # w from MLE
        energy0 = -ops.dot(w_mle)    
        # select the best sequence that maximize probability
        s_hidden_recover0 = s_hidden_possibles[np.argmin(energy0)]
        #print(s_hidden_recover)
        acc[0,t] = np.sum((s[hidden] == s_hidden_recover0))
        #print(acc[t])
        """
        #----------------------------------------------
        # w from PLE
        energy1 = -ops.dot(w_ple)    
        s_hidden_recover1 = s_hidden_possibles[np.argmin(energy1)]
        acc[1,t] = np.sum((s[hidden] == s_hidden_recover1))

        #----------------------------------------------
        # w from EM
        energy2 = -ops.dot(w_em)
        s_hidden_recover2 = s_hidden_possibles[np.argmin(energy2)]
        acc[2,t] = np.sum((s[hidden] == s_hidden_recover2))

    acc_av = acc.sum(axis=1)/(n_hidden*l) 
    print('accuracy of MLE, PLE, EM:',n_hidden,acc_av)

accuracy of MLE, PLE, EM: 2 [0.         0.94872958 0.84392015]
accuracy of MLE, PLE, EM: 4 [0.         0.92785844 0.84618875]
accuracy of MLE, PLE, EM: 6 [0.         0.88309135 0.84497883]
accuracy of MLE, PLE, EM: 8 [0.         0.8384755  0.84108439]
accuracy of MLE, PLE, EM: 10 [0.         0.78502722 0.8331216 ]
accuracy of MLE, PLE, EM: 12 [0.         0.73911071 0.83136721]
